## This is a notebook for computing MPF using AMSR-E brightness temperature

author: Kexin Song
Date: 09/05/2019

In [ ]:
import numpy as np
# from pyhdf import SD
import xarray as xr
import glob

In [3]:
from pydap.client import open_url
from pydap.cas.urs import setup_session
url = 'https://acdisc.gesdisc.eosdis.nasa.govacdis/opendap/HDF-EOS5/Aura_MLS_Level2/ML2BRO.004/2016/MLS-Aura_L2GP-BrO_v04-23-c03_2016d303.he5'
session = setup_session('Kexin.Song', 'Kathy-mao3gou4') # Use yours here.
url = 'https://acdisc.gesdisc.eosdis.nasa.govacdis/opendap/HDF-EOS5/Aura_MLS_Level2/ML2BRO.004/2016/MLS-Aura_L2GP-BrO_v04-23-c03_2016d303.he5'
dataset = open_url(url, session=session)

ConnectionError: HTTPSConnectionPool(host='acdisc.gesdisc.eosdis.nasa.govacdis', port=443): Max retries exceeded with url: /opendap/HDF-EOS5/Aura_MLS_Level2/ML2BRO.004/2016/MLS-Aura_L2GP-BrO_v04-23-c03_2016d303.he5.dds (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f38e68236a0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [ ]:
from pydap.client import open_url
from pydap.cas.urs import setup_session

url = 'https://n5eil01u.ecs.nsidc.org/AMSA/AE_SI25.003/2008.07.01/AMSR_E_L3_SeaIce25km_V15_20080701.hdf.xml'

session = setup_session(username = 'Kexin.Song', password = 'Kathy-mao3gou4', check_url=url)

ds = open_url(url, session=session)
ds

In [ ]:
fpath = "C:\\Users\\kathy\\research\\Sea-ice-concentration\\data\\Melt-Pond-Fraction\\AMSR-E\\"
fname = "AMSR_E_L3_SeaIce12km_V15_20080701.hdf"
f = fpath+fname
ds = xr.open_dataset(f)
ds

In [ ]:
''' Read TB from hdf file'''
def ReadTB(filename):
   
    hdf_name = glob.glob(filename)
    hdf_obj = SD.SD(hdf_name[0],SD.SDC.READ)
#    key = hdf_obj.datasets().keys()
#    print (key)
    TB89V = list(hdf_obj.select('SI_12km_NH_89V_DAY'))  # SDS强制类型转换为list   
    return TB89V	

''' Read Land mask from hdf file'''
def ReadLM(filename):
   
    hdf_name = glob.glob(filename)
    hdf_obj = SD.SD(hdf_name[0],SD.SDC.READ)
    lm = np.array(list(hdf_obj.select('landmask')))  # SDS强制类型转换为list   
    return lm

## Read landmask 12km
fpath = "C:\\Users\\kathy\\research\\Sea-ice-concentration\\data\\Melt-Pond-Fraction\\AMSR-E\\mask\\"
fname = "amsr_gsfc_12n.hdf"
f = fpath + fname
lm12 = ReadLM(f)

## Read landmask 25km
fpath = "C:\\Users\\kathy\\research\\Sea-ice-concentration\\data\\Melt-Pond-Fraction\\AMSR-E\\mask\\"
fname = "amsr_gsfc_25n.hdf"
f = fpath + fname
lm25 = ReadLM(f)




## Read TB89V
fpath = "C:\\Users\\kathy\\research\\Sea-ice-concentration\\data\\Melt-Pond-Fraction\\AMSR-E\\"
fname = "AMSR_E_L3_SeaIce12km_V15_20080701.hdf"

f = fpath+fname
hdf_obj = SD.SD(f,SD.SDC.READ)
#keys = hdf_obj.datasets().keys()
#print(keys)
TB89V = np.array(list(hdf_obj.select('SI_12km_NH_89V_DAY')))
TB89Vm = np.ma.MaskedArray(TB89V,lm12)


## Read TB06H
fname = "AMSR_E_L3_SeaIce25km_V15_20080701.hdf"

f = fpath+fname
hdf_obj = SD.SD(f,SD.SDC.READ)
TB06H = np.array(list(hdf_obj.select('SI_25km_NH_06H_DAY')))
TB06Hm = np.ma.MaskedArray(TB06H,lm25)

## rebin TB89Vm to 25km
import rebin
TB89Vr = rebin.rebin(TB89Vm,(2,2))

## mask TB89Vr
TB89Vrm = np.ma.MaskedArray(TB89Vr,lm25)

print(TB89Vm.mean(),TB89Vrm.mean())


## calculate MPF
add = TB06H + TB89Vrm
dif = TB06H - TB89Vrm
mpf = 15.2 - (158.9*dif/add)

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

## plot mpf
plt.figure(figsize=(8,8))
#mpf[mpf<0] = np.nan
#mpfm = np.ma.masked_invalid(mpf)
plt.imshow(mpf)
#plt.imshow(mpfm)
plt.colorbar()

## plot 1-mpf
plt.figure(figsize=(8,8))
plt.imshow(100-mpf,cmap="gist_ncar")
plt.colorbar()

## plot absolute mpf = mpf/95 (assume sic=95%)
mpfa = mpfm/95*100
plt.figure(figsize=(8,8))
plt.imshow(mpfa)
plt.colorbar()

## plot 1- absolute mpf
plt.figure(figsize=(8,8))
plt.imshow(100-mpfa,cmap="gist_ncar")
plt.colorbar()

## plot (1-mpf)-(1-absolute mpf)
plt.figure(figsize=(8,8))
d = mpfa-mpf
print(d.min(),d.mean(),d.max())

d[d>2] = np.nan
dm = np.ma.masked_invalid(d)

plt.imshow(dm)
plt.colorbar()